# Linear and Polynomial Regression

## Linear Regreesion Using ScikitLearn <a class="anchor" id="scikit"></a>
 <a class="anchor" id="LinearRegression"></a>

## Imports

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# inline plot
%matplotlib inline
# default figure size
matplotlib.rcParams['figure.figsize'] = (20, 10)
# to make our sets reproducible
np.random.seed(42)
# %config Completer.use_jedi = False

In [ ]:
from typing import List, Dict, Tuple, Union, Optional, Callable

### Data generation <a class="anchor" id="linear_data_generation"></a>

$$Y = 3 \cdot X + 4 + \epsilon | \epsilon \sim  N(0,1)$$

In [ ]:
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

## Linear Regreesion Using ScikitLearn

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y)  # Trying to solve the (X^TX)^-1X^Ty PROBLEM

In [ ]:
# Parameters
print("Intercept:", lin_reg.intercept_[0])
print("Slope:", lin_reg.coef_[0][0])


### Converging with different learning rates

In [ ]:
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)
X_new = 2 * np.random.rand(20, 1)

X_b = np.c_[np.ones((100, 1)), X]
X_new_b = np.c_[np.ones((20, 1)), X_new]

def plot_gradient_descent(theta, eta:float, theta_path=None, n_iterations: int = 1000):
    # number of samples
    m = len(X_b)
    # plot the samples with their corrseponding labels
    plt.plot(X, y, "b.")

    for iteration in range(n_iterations):

        if iteration < 10:
            # make predictions
            y_predict = X_new_b.dot(theta)
            style = "b-" if iteration > 0 else "r--"
            # plot the predictions
            plt.plot(X_new, y_predict, style)

        # gradient is computed using all datapoints
        gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradients
        # add new
        if theta_path is not None:
            theta_path.append(theta)

    plt.xlabel("$x_1$", fontsize=18)
    plt.axis([0, 2, -5, 15])
    plt.title(r"$\eta = {}$".format(eta), fontsize=16)

theta_path_bgd = []

np.random.seed(42)
theta = np.random.randn(2,1)  # random initialization

plt.figure(figsize=(14,6))
plt.subplot(131); plot_gradient_descent(theta, eta=0.02)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.subplot(132); plot_gradient_descent(theta, eta=0.1, theta_path=theta_path_bgd)
plt.subplot(133); plot_gradient_descent(theta, eta=0.5)


## Stochastic Gradient Descent using scikit  <a class="anchor" id="sgd"></a>

In [ ]:
# SGD = Stochastic Gradient Descent
# The model updates its parameters after each sample (or mini-batch),
# using a learning rate schedule to gradually reduce update strength.

from sklearn.linear_model import SGDRegressor

# Create and train the SGD regressor
sgd_reg = SGDRegressor(
    max_iter=1000,      # number of epochs (passes over data)
    eta0=0.1,           # initial learning rate
    learning_rate='constant',  # keep learning rate fixed
)

sgd_reg.fit(X, y.ravel())

# Print learned parameters
print(f"Intercept: {sgd_reg.intercept_[0]:.6f}")
print(f"Slope: {sgd_reg.coef_[0]:.6f}")


## Polynomial Regression <a class="anchor" id="polynomial_regression"></a>

In [ ]:
m = 100
X = 6 * np.random.rand(m, 1) - 3  # -3 <= x <= 3
y = 0.5 * X**2 - 0.5 * X + 2 + np.random.randn(m, 1) / 4

# test data
X_test = 6 * np.random.rand(30, 1) - 3
y_test = 0.5 * X_test**2 - 0.5 * X_test + 2 + np.random.randn(30, 1) / 4


Generate polynomial features (degree 2)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)  # [x] -> [x, x**2]


In [ ]:
print(f""" data:
{X[:5]}

data with deg 2 polynomial features:

{X_poly[:5]}
""")

In [ ]:
## fitting  linear regression
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)

print(f'''
features coefficients: {lin_reg.coef_}
intercept: {lin_reg.intercept_}
''')

## Evaluation
from sklearn.metrics import mean_squared_error

# train predictions
train_predictions = lin_reg.predict(X_poly)
train_mse = mean_squared_error(y, train_predictions)

# evaluation prediction
predictions = lin_reg.predict(poly_features.transform(X_test))
mse = mean_squared_error(y_test, predictions)

rmse, train_rmse = np.sqrt(mse), np.sqrt(train_mse)
print(f"""
    train RMSE: {train_rmse}
    test RMSE: {rmse}
""")

from sklearn.metrics import r2_score
train_r2 = r2_score(y_true=y,y_pred=train_predictions)
r2 = r2_score(y_true=y_test,y_pred=predictions)
print(f"""
    train R2: {train_r2}
    test R2: {r2}
""")

In [ ]:
plt.figure(figsize=(10, 6))
# plot train and test
plt.plot(X_test, y_test, "go", label='test')
plt.plot(X, y, "b.", label='train')

# plot the polynomial
xs = np.linspace(-3, 3, 20)[:, np.newaxis]
ys = lin_reg.predict(poly_features.transform(xs))
plt.plot(xs, ys, "r-", label='fit')

plt.axis([-2, 3, 1, 6])
plt.legend()
plt.show()

## Learning Curves <a class="anchor" id="learning_curves"></a>

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# split test as the largest X values (sorted), keep train as the rest
def split_sorted(X: np.ndarray, y: np.ndarray, test_size: float):
    idxs = np.argsort(X.reshape(-1))
    n_test = int(len(y) * test_size)
    if n_test == 0:
        raise ValueError("test_size too small for dataset size")
    train_idxs = idxs[:-n_test]
    test_idxs = np.setdiff1d(np.arange(len(y)), train_idxs)
    return X[train_idxs], X[test_idxs], y[train_idxs], y[test_idxs]

def plot_learning_curves(model, X, y, split_sort: bool = False):
    # split to train and test
    if split_sort:
        X_train, X_val, y_train, y_val = split_sorted(X, y, test_size=0.2)
    else:
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)

    train_errors, val_errors = [], []

    # increase the train sequentially
    for m in range(1, len(X_train) + 1):
        model.fit(X_train[:m], y_train[:m])
        y_train_predict = model.predict(X_train[:m])
        y_val_predict = model.predict(X_val)
        train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
        val_errors.append(mean_squared_error(y_val, y_val_predict))

    # one figure with two subplots
    fig, axes = plt.subplots(ncols=2, figsize=(8, 3))

    # left: data + current fit
    axes[0].plot(X_train, y_train, "r.", label="train")
    axes[0].plot(X_val, y_val, "bo", label="validation")
    xs = np.linspace(X.min(), X.max(), 200).reshape(-1, 1)
    ys = model.predict(xs).ravel()
    axes[0].plot(xs, ys, "g-", label="fit")
    axes[0].legend(loc="upper right", fontsize=10)

    # right: learning curves (RMSE)
    axes[1].plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
    axes[1].plot(np.sqrt(val_errors), "b-", linewidth=2, label="val")
    axes[1].legend(loc="upper right", fontsize=10)
    axes[1].set_xlabel("Training set size")
    axes[1].set_ylabel("RMSE")

    plt.tight_layout()
    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# -------------------------
# 1) Data
# -------------------------
np.random.seed(42)
m = 50
X = np.linspace(-3, 3, m).reshape(-1, 1)  # -3 <= x <= 3
y = 0.5 * X**2 + X + 2 + 0.3 * np.random.randn(m, 1)

# grid for smooth curves
xs = np.linspace(X.min(), X.max(), 400).reshape(-1, 1)

# -------------------------
# 2) Fit models and compute BIC
# -------------------------
def fit_poly_and_bic(X, y, degree):
    """Fit polynomial regression of given degree; return model and BIC."""
    model = make_pipeline(
        PolynomialFeatures(degree=degree, include_bias=True),  # include_bias=True -> intercept handled in features
        LinearRegression(fit_intercept=False)                  # so we avoid double-intercept
    )
    model.fit(X, y)
    y_hat = model.predict(X)

    # Residual sum of squares
    rss = float(np.sum((y - y_hat)**2))
    n = len(y)

    # Number of parameters = number of polynomial terms up to 'degree'
    # With include_bias=True in PolynomialFeatures, k = degree + 1 (for 1D X)
    k = degree + 1

    # Gaussian-error BIC: n*ln(RSS/n) + k*ln(n)
    bic = n * np.log(rss / n) + k * np.log(n)
    return model, bic, rss

max_degree = 15
models = {}
bics = []
rss_list = []

for deg in range(1, max_degree + 1):
    model, bic, rss = fit_poly_and_bic(X, y, deg)
    models[deg] = model
    bics.append(bic)
    rss_list.append(rss)

degrees = np.arange(1, max_degree + 1)
bics = np.array(bics)

best_deg = int(degrees[np.argmin(bics)])

# Choose canonical examples
under_deg = 1
over_deg = max_degree
good_deg = best_deg

print(f"Best degree by BIC: {best_deg}")

# -------------------------
# 3) Plot underfit vs good fit vs overfit
# -------------------------
fig, axes = plt.subplots(1, 3, figsize=(12, 3), sharey=True)
cases = [("Underfit", under_deg), ("Good (BIC-best)", good_deg), ("Overfit", over_deg)]

for ax, (title, deg) in zip(axes, cases):
    ax.scatter(X, y, s=18, alpha=0.7, label="data")
    ys = models[deg].predict(xs).ravel()
    ax.plot(xs, ys, lw=2, label=f"degree={deg}")
    ax.set_title(title)
    ax.set_xlabel("x")
    ax.grid(alpha=0.2)
axes[0].set_ylabel("y")
axes[1].legend(loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
# -------------------------
# 4) BIC “elbow” plot
# -------------------------
plt.figure(figsize=(5, 3))
plt.plot(degrees, bics, marker="o")
plt.xlabel("Polynomial degree")
plt.ylabel("BIC (lower is better)")
plt.title("Model selection by BIC")
plt.grid(alpha=0.3)
# mark best
plt.scatter([best_deg], [bics[best_deg - 1]], s=80, zorder=3)
plt.annotate(f"best = {best_deg}", xy=(best_deg, bics[best_deg - 1]),
             xytext=(best_deg + 0.5, bics[best_deg - 1] + 0.5),
             arrowprops=dict(arrowstyle="->", lw=1))
plt.tight_layout()
plt.show()


## Regularized regression <a class="anchor" id="regularization"></a>

### Ridge <a class="anchor" id="ridge_lasso_elastic"></a>

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1)
ridge_reg.fit(X, y)
ridge_reg.predict([[1.5]])

### Using different regularization parameter

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, LinearRegression
from typing import Callable, List, Dict

# Generate data
np.random.seed(42)
m = 50
X = np.linspace(-3, 3, m).reshape(-1, 1)
y = 0.5 * X**2 + X + 2 + 0.5 * np.random.randn(m, 1)

# Utility function
def plot_model(model_class: Callable, polynomial: bool, alphas: List[float], title: str, **model_kargs: Dict):
    """
    Show the effect of regularization (alpha) for linear or polynomial models.
    """
    X_grid = np.linspace(X.min(), X.max(), 300).reshape(-1, 1)
    styles = ("b-", "g--", "r:")
    lw_for = lambda a: 2 if a > 0 else 1

    for alpha, style in zip(alphas, styles):
        base = model_class(alpha=alpha, **model_kargs)
        if polynomial:
            model = Pipeline([
                ("poly", PolynomialFeatures(degree=10, include_bias=False)),
                ("scaler", StandardScaler()),
                ("ridge", base)
            ])
        else:
            model = base

        model.fit(X, y)
        y_pred = model.predict(X_grid)
        plt.plot(X_grid, y_pred, style, linewidth=lw_for(alpha),
                 label=fr"$\alpha = {alpha}$")

    plt.scatter(X, y, color="black", s=20, alpha=0.6, label="data")
    plt.title(title)
    plt.xlabel("$x$")
    plt.ylabel("$y$")
    plt.legend(fontsize=10)
    plt.grid(alpha=0.25)



In [ ]:
plt.subplot(1, 2, 1)
plot_model(Ridge, polynomial=False, alphas=(0, 10, 100),
            title="Linear (Underfit)")

plt.subplot(1, 2, 2)
plot_model(Ridge, polynomial=True, alphas=(0, 1e-3, 1),
            title="Polynomial (Regularization Effect)")

plt.tight_layout()
plt.show()


## Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X, y)
lasso_reg.predict([[1.5]])

In [ ]:
from sklearn.linear_model import Lasso

plt.subplot(121)
plot_model(Lasso,polynomial=False,alphas=(1e-7, 0.1, 1),title="Linear (Lasso)",random_state=42)
plt.ylabel("$y$", rotation=0, fontsize=18)

plt.subplot(122)
plot_model(Lasso,polynomial=True,alphas=(1e-7, 1e-3, 1),title="Polynomial (Lasso)",random_state=42)


## Elatic Net

In [ ]:
from sklearn.linear_model import ElasticNet

# ElasticNet do alpha(rho||theta||_1 + (1-rho)||theta||_2^2 /2) so l1_ratio = rho
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
elastic_net.fit(X, y)
elastic_net.predict([[1.5]])

In [ ]:
from sklearn.linear_model import ElasticNet

plt.subplot(121)
plot_model(ElasticNet,polynomial=False,alphas=(1e-7, 0.1, 1),title="Linear (ElasticNet)",random_state=42, l1_ratio=0.5)
plt.ylabel("$y$", rotation=0, fontsize=18)

plt.subplot(122)
plot_model(ElasticNet,polynomial=True,alphas=(1e-7, 1e-3, 1),title="Polynomial (ElasticNet)",random_state=42, l1_ratio=0.5)


## Early Stopping <a class="anchor" id="early_stopping"></a>

In [ ]:
np.random.seed(42)
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 2 + X + 0.5 * X**2 + np.random.randn(m, 1)

X_train, X_val, y_train, y_val = train_test_split(X[:50], y[:50].ravel(), test_size=0.5, random_state=10)


In [ ]:
from copy import deepcopy

# feature engineering
poly_scaler = Pipeline([  # very important to set include_bias=False, because SGRegressor and LinearRegression add the intercept
        ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
        ("std_scaler", StandardScaler())
    ])

# train and transform on training set
X_train_poly_scaled = poly_scaler.fit_transform(X_train)
# only transform on test
X_val_poly_scaled = poly_scaler.transform(X_val)

# configure for one iteration only (one epoch) - so we can go over epochs manually
sgd_reg = SGDRegressor(max_iter=1, tol=None, warm_start=True,  # max_iter: take one epoch, tol: no tolerance to stop algorithm
                       penalty=None, learning_rate="constant", eta0=0.0005, random_state=42)

# warm_start=True means that each fit will use the previous learned fit

minimum_val_error = float("inf")
best_epoch = None
best_model = None
for epoch in range(1000):
    sgd_reg.fit(X_train_poly_scaled, y_train)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)  # sgd.step()
    val_error = mean_squared_error(y_val, y_val_predict)
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = deepcopy(sgd_reg)
print('best model')
print(best_model.coef_, best_model.intercept_)
print()
print(f"best epoch: {best_epoch}")

In [ ]:
sgd_reg = SGDRegressor(max_iter=1, tol=None, warm_start=True,
                       penalty=None, learning_rate="constant", eta0=0.0005, random_state=42)

n_epochs = 500
train_errors, val_errors = [], []
for epoch in range(n_epochs):
    sgd_reg.fit(X_train_poly_scaled, y_train)
    y_train_predict = sgd_reg.predict(X_train_poly_scaled)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    train_errors.append(mean_squared_error(y_train, y_train_predict))
    val_errors.append(mean_squared_error(y_val, y_val_predict))

best_epoch = np.argmin(val_errors)
best_val_rmse = np.sqrt(val_errors[best_epoch])

plt.annotate('Best model',
             xy=(best_epoch, best_val_rmse),
             xytext=(best_epoch, best_val_rmse + 1),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=16,
            )

best_val_rmse -= 0.03  # just to make the graph look better
plt.plot([0, n_epochs], [best_val_rmse, best_val_rmse], "k:", linewidth=2)  # the dotted line
plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="Validation set")
plt.plot(np.sqrt(train_errors), "r--", linewidth=2, label="Training set")
plt.legend(loc="upper right", fontsize=14)
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("RMSE", fontsize=14)
